In [1]:
# مشروع اكتشاف التسلل مع تحديد خط التسلل واللاعب المتسلل
import cv2
import numpy as np
from ultralytics import YOLO
import easyocr

# تحميل النموذج
model = YOLO('yolov8n.pt')
reader = easyocr.Reader(['en'])

cap = cv2.VideoCapture('offside_example.mp4')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('offside_detection_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    results = model(frame)[0]
    boxes = results.boxes
    players = []

    for box in boxes:
        cls_id = int(box.cls[0])
        if cls_id == 0:  # person
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx = (x1 + x2) // 2
            cy = y2  # أسفل اللاعب
            players.append((cx, cy, (x1, y1, x2, y2)))

    # التحقق من وجود أكثر من لاعب
    if len(players) >= 3:
        players_sorted = sorted(players, key=lambda x: x[0])  # من اليسار لليمين
        defender_line_x = players_sorted[-2][0]  # آخر ثاني مدافع
        cv2.line(frame, (defender_line_x, 0), (defender_line_x, height), (0, 255, 255), 2)

        for (cx, cy, (x1, y1, x2, y2)) in players:
            if cx > defender_line_x:  # المهاجم خلف خط آخر ثاني مدافع
                player_crop = frame[y1:y2, x1:x2]
                gray_crop = cv2.cvtColor(player_crop, cv2.COLOR_BGR2GRAY)
                result = reader.readtext(gray_crop)
                player_number = '?'
                for (_, text, _) in result:
                    if text.isdigit():
                        player_number = text
                        break
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, f'Offside! Player #{player_number}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
                cv2.putText(frame, 'Offside Detected', (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    out.write(frame)

cap.release()
out.release()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



0: 352x640 15 persons, 395.3ms
Speed: 52.7ms preprocess, 395.3ms inference, 41.5ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 15 persons, 162.9ms
Speed: 6.3ms preprocess, 162.9ms inference, 2.4ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 15 persons, 148.7ms
Speed: 4.7ms preprocess, 148.7ms inference, 1.8ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 16 persons, 158.8ms
Speed: 5.0ms preprocess, 158.8ms inference, 1.2ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 14 persons, 154.1ms
Speed: 4.4ms preprocess, 154.1ms inference, 3.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 15 persons, 185.0ms
Speed: 4.2ms preprocess, 185.0ms inference, 1.9ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 14 persons, 203.0ms
Speed: 6.9ms preprocess, 203.0ms inference, 1.9ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 14 persons, 150.2ms
Speed: 6.7ms preprocess, 150.2ms inference, 1.8ms postpr